<a href="https://www.kaggle.com/code/yassinezarwal/params-tuning-s-m-housing?scriptVersionId=96691181" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer


data_train=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv',index_col=['Id'])
data_test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv',index_col=['Id'])


# data split

train_x=data_train.drop(columns=['SalePrice'])
test_x=data_test
train_y=data_train.pop('SalePrice')

# categorical vs numeric data
ncolumns=train_x.select_dtypes(['int','float']).columns.tolist()
ccolumns=train_x.select_dtypes(['object']).columns.tolist()
train_x=train_x[ncolumns+ccolumns]


# cleaning data
cpipeline=Pipeline([('cimpute',SimpleImputer(strategy='most_frequent')),
                                        ('cencoder',OrdinalEncoder())])
npipeline=Pipeline([('nimpute',SimpleImputer(strategy='mean'))])

full_pipeline=ColumnTransformer([('num',npipeline,ncolumns),('cat',cpipeline,ccolumns)])


train_x=pd.DataFrame(full_pipeline.fit_transform(train_x),index=train_x.index,columns=train_x.columns)
test_x=pd.DataFrame(full_pipeline.fit_transform(test_x),index=test_x.index,columns=test_x.columns)

index_test=test_x.index

# scaling data
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

train_x=scaler.fit_transform(train_x)
test_x=scaler.fit_transform(test_x)



import tensorflow as tf 
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression

import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV


# split data
train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y)


# Build a model
def build_model(n_hidden=1,n_neurons=30,learning_rate=0.01,input_shape=train_x.shape[1:]):
    model=keras.models.Sequential()
    options={'input_shape':input_shape}

    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='relu',**options))
        options={}

    model.add(keras.layers.Dense(1,**options))
    optimizer=keras.optimizers.SGD(learning_rate)
    model.compile(loss='mse',optimizer=optimizer)
    return model


keras_reg=keras.wrappers.scikit_learn.KerasRegressor(build_model)


# hyperparametrs tuning
a=np.arange(2,40)
b=np.arange(0.05,0.1,0.005)
param_distribs={
    'n_neurons':a.tolist(),
    'n_hidden':[2,3,4],
    'learning_rate':b.tolist(),
}


rnd_search_cv=RandomizedSearchCV(keras_reg,param_distribs,n_iter=50,cv=3)
rnd_search_cv.fit(train_x,train_y,epochs=150,
                  validation_data=(valid_x,valid_y),callbacks=[ keras.callbacks.EarlyStopping(patience=6)])


print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)
model=rnd_search_cv.best_estimator_.model

# make prediction for test_data
predictions=model.predict((test_x))

predictions=pd.DataFrame(predictions,index=index_test,columns=['SalePrice'])

predictions.to_csv('gender_submission_housing32.csv')

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


2022-05-26 19:37:50.727249: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-05-26 19:37:50.727411: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-26 19:37:56.111797: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-26 19:37:56.114787: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-05-26 19:37:56.114835: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-26 19:37:56.114867: I tensorflow/stream_executor/cuda/cud

Epoch 1/150
23/23 [==============================] - 1s 29ms/step - loss: nan - val_loss: nan
Epoch 2/150
23/23 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 3/150
23/23 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 4/150
23/23 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 5/150
23/23 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 6/150
12/12 [==============================] - 0s 1ms/step - loss: nan
Epoch 1/150
23/23 [==============================] - 0s 10ms/step - loss: nan - val_loss: nan
Epoch 2/150
23/23 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 3/150
23/23 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 4/150
23/23 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 5/150
23/23 [==============================] - 0s 4ms/step - loss: nan - val_l

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: RuntimeWarning: invalid value encountered in subtract
  array_means[:, np.newaxis]) ** 2,


35/35 [==============================] - 0s 7ms/step - loss: 2536392051953981397663744.0000 - val_loss: 65919398852054482944.0000
Epoch 2/150
35/35 [==============================] - 0s 3ms/step - loss: 21814658339273150464.0000 - val_loss: 8566474406887424.0000
Epoch 3/150
35/35 [==============================] - 0s 3ms/step - loss: 2834832312762368.0000 - val_loss: 1129565519872.0000
Epoch 4/150
35/35 [==============================] - 0s 3ms/step - loss: 376760228067.5555 - val_loss: 6400011776.0000
Epoch 5/150
35/35 [==============================] - 0s 3ms/step - loss: 5973288945.7778 - val_loss: 6244018688.0000
Epoch 6/150
35/35 [==============================] - 0s 3ms/step - loss: 6445869425.7778 - val_loss: 6240983040.0000
Epoch 7/150
35/35 [==============================] - 0s 3ms/step - loss: 6443637034.6667 - val_loss: 6234150400.0000
Epoch 8/150
35/35 [==============================] - 0s 3ms/step - loss: 6647425351.1111 - val_loss: 6232977920.0000
Epoch 9/150
35/35 [=====

In [2]:
print(predictions)

          SalePrice
Id                 
1461  181531.390625
1462  181531.390625
1463  181531.390625
1464  181531.390625
1465  181531.390625
...             ...
2915  181531.390625
2916  181531.390625
2917  181531.390625
2918  181531.390625
2919  181531.390625

[1459 rows x 1 columns]
